# Extração e seleção de características

As características que são extraídas dos dados dependem de sua natureza. Os dados EMG são sinais elétricos coletados dentro de um período de tempo, portanto são dados no domínio do tempo. As características deste domínio são extraídas dele. Também é possível transformar os dados EMG para o domínio da frequência e extrair características neste domínio. Existem diversas características que podem ser extraídas de cada domínio, entretando nem todas elas serão relevantes. Cada problema se beneficia de características ou combinações delas. Portanto, é preciso que haja uma seleção de características para encontrar a combinação de características que trará melhor resultado na posterior classificação dos dados.

## Reutilizando os passos anteriores

É necessário carregar os dados pré-processados, para dar início à extração de características. No jupyter notebook podemos utilizar o namespace completo de outro notebook:

In [4]:
%run preprocessing.ipynb

(20000, 2)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 1
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 2
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 3
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 4
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 5
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 6
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 7
(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)
(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)----- 

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em EMG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) \\$
    $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn([x_i - threshold][x_{i+1} - threshold]) \\$
    $ sgn(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $
    $Sugestão: threshold = 0.4$

### Domínio da frequência

1. Auto Regressive (AR)

    > $ - \sum_{j=1}^{\rho}\alpha_j x_{j-1} + w_n $

2. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

3. Mean Frequency (FMN)

    > $ \sum_{j=1}^{M}f_j PSD_j \Big{ / } \sum_{j=1}^{M}PSD_j $

4. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{i=1}^{M}|w|_i $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Desafio 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

### Extraindo características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características VAR & RMS (domínio do tempo) e FDM & MMDF (domínio da frequência).

In [70]:
from math import prod

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def f_j(j, SampleRate, M):
    return (j * SampleRate) / (2 * M)

def A_j(w):
    return np.abs(w)


# funções de extração de características


def WAMP(x, limiar):
    return np.sum( np.abs(np.diff(x)) > limiar, axis= -1)

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def WL(x):
    return np.sum(np.abs(np.diff(x)), axis = -1)

def ZC_Add(data, th):
    
    somatorio = 0
    resultado = 0
    tamanho = len(data)
    
    for i in range(tamanho - 1):
        resultado1 = (data[i] * data[i+1])
        resultado2 = np.abs((data[i] - data[i+1]))
        
        if(resultado1 < 0) and (resultado2 >=  th):
            somatorio += 1
    
    return somatorio

def FMD(w):
    return np.sum(PSD(w), axis=-1) / 2

def FMN_Add(data):
    M = len(data)
    somatorio = 0
    
    for j in range(M):
        somatorio += (f_j(j, 41, M) * PSD(data)) / np.sum(PSD(data))
    
    return somatorio

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def MMNF_Add(w):
    M = len(data)
    somatorio = 0
    
    for j in range(M):
        somatorio += (f_j(j, 41, M) * A_j(data)) / np.sum(A_j(data))
    
    return somatorio

In [71]:
def FMN(data):
    x,y,z = data.shape[:3]
    somatorio_final = []
    
    for i in range(x):
        somatorio_fx = []
        for j in range(y):
            somatorio_fy = []
            for k in range(z):
                somatorio_fz = FMN_Add(data[i][j][k])
                
                somatorio_fy.append(somatorio_fz)
            
            somatorio_fx.append(somatorio_fy)
        
        somatorio_final.append(somatorio_fx)
        
    return np.array(somatorio_final)

# MMNF

In [72]:
def MMNF(data):
    x,y,z = data.shape[:3]
    somatorio_final = []
    
    for i in range(x):
        somatorio_fx = []
        for j in range(y):
            somatorio_fy = []
            for k in range(z):
                somatorio_fz = MMNF_Add(data[i][j][k])
                
                somatorio_fy.append(somatorio_fz)
            
            somatorio_fx.append(somatorio_fy)
        
        somatorio_final.append(somatorio_fx)
        
    return np.array(somatorio_final)

In [ ]:
MMNF(chunks_freq)

**Desafio 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características EMG além das apresentadas, citando as fontes de tais características.


## Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

## Implementação do vetor

In [8]:
chunks_time.shape

(60, 2, 41, 1024)

In [12]:
PontoMax = np.max(chunks_time)
PontoMax

0.001966231296292373

In [13]:
Mediana = np.median(chunks_time)
Mediana


2.4288825703437275e-06

In [14]:
MeuLimiar = (PontoMax + Mediana) / 2
MeuLimiar

0.0009843300894313584

In [32]:
Media = np.mean(chunks_time)
Media

-1.3098920532612888e-08

In [34]:
MeuLimiarNovo = (PontoMax + Media) / 2
MeuLimiarNovo

0.0009831090986859204

In [19]:
chunks_time.shape

(60, 2, 41, 1024)

In [57]:
def ZC(data, th):
    
    x,y,z = data.shape[:3]
    somatorio_final = []
    for i in range(x):
        somatorio_fx = []
        for j in range(y):
            somatorio_fy = []
            for k in range(z):
                somatorio_fz = ZC_Add(data[i][j][k], th)
                
                somatorio_fy.append(somatorio_fz)
            
            somatorio_fx.append(somatorio_fy)
        
        somatorio_final.append(somatorio_fx)
        
    return np.array(somatorio_final)

In [58]:
ZC(chunks_time, Mediana)


array([[[105, 104, 109, ..., 108, 106, 104],
        [140, 133, 131, ..., 118, 131, 125]],

       [[104, 103, 110, ...,  90,  94,  98],
        [120, 125, 126, ..., 117, 117, 122]],

       [[100,  94,  93, ..., 105, 110, 100],
        [131, 130, 128, ..., 119, 119, 114]],

       ...,

       [[ 93,  85,  88, ..., 101,  96,  86],
        [101, 104, 101, ..., 102, 106, 117]],

       [[103,  99,  85, ...,  98,  95,  89],
        [105, 100,  92, ..., 105,  99, 100]],

       [[ 90,  98, 101, ..., 104,  94,  89],
        [113, 108,  97, ...,  94,  92, 100]]])

In [37]:
WAMP(chunks_time, Mediana)

array([[[ 965,  957,  967, ...,  986,  988,  983],
        [ 878,  898,  914, ...,  934,  942,  932]],

       [[ 984,  989,  980, ...,  970,  982,  986],
        [ 923,  945,  958, ...,  927,  920,  915]],

       [[ 939,  960,  963, ...,  984,  986,  985],
        [ 840,  878,  904, ...,  950,  929,  927]],

       ...,

       [[ 994,  988,  989, ...,  990,  992, 1003],
        [ 972,  977,  975, ...,  970,  968,  956]],

       [[ 963,  965,  982, ...,  992,  990,  987],
        [ 947,  951,  966, ...,  945,  955,  955]],

       [[ 945,  974,  985, ...,  991,  994,  999],
        [ 915,  941,  956, ...,  943,  980,  982]]])

In [59]:
final_data = list()

final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))
final_data.append(WAMP(chunks_time, Mediana))
final_data.append(WL(chunks_time))
final_data.append(ZC(chunks_time, 0))

final_data.append(fmd(chunks_freq))
final_data.append(mmdf(chunks_freq))


final = np.array(final_data)
final.shape

(7, 60, 2, 41)

É necessário que seja reordenado as dimensões do vetor de características, pois cada característica (de cada canal), deve corresponder à última dimensão do vetor. Por fim, as outras dimensões são concatenadas para o número de amostras.

In [4]:
data = final.transpose(1, 3, 2, 0)
X = data.reshape(data.shape[0]*data.shape[1], data.shape[2]*data.shape[3])
X.shape

(2460, 8)

## Seleção de características

Nesta etapa, são selecionadas as características que mais afetam positivamente no resultado final da classificação. Vamos estudar os métodos de seleção de características nesta [página do projeto sklearn](https://scikit-learn.org/stable/modules/feature_selection.html).

**Desafio 3**: mostrar o resultado para os dados de trabalho, para os seguintes métodos se leção de características:
- VarianceThreshold
- Univariate feature selection
    - escolha o que mais for "interessante": `SelectKBest`, `SelectPercentile` e `GenericUnivariateSelect`
- Recursive feature elimination